### show FID

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os

save_disk = "D:/TF_LOGS"
k = 3

paths_pix2pix = [
    ["Alizarine", "20230323-110848", 0],
    ["Alizarine", "20230327-224625", 1],
    ["Alizarine", "20230328-021427", 2],
    ["Gavet", "20230323-120633", 0],
    ["Gavet", "20230327-234144", 1],
    ["Gavet", "20230328-030749", 2],
    ["Rotterdam_1000", "20230323-132240", 0],
    ["Rotterdam_1000", "20230328-004342", 1],
    ["Rotterdam_1000", "20230328-095439", 2],
]

paths_our = [
    ["Alizarine", "20220405-2359", 0],
    ["Alizarine", "20220603-1720", 1],
    ["Alizarine", "20220603-1850", 2],
    ["Gavet", "20220429-0021", 0],
    ["Gavet", "20220609-2213", 1],
    ["Gavet", "20220609-2319", 2],
    ["Rotterdam_1000", "20220623-0916", 0],
    ["Rotterdam_1000", "20220609-1841", 1],
    ["Rotterdam_1000", "20220609-2031", 2],
    ["Rotterdam_1000", "20230330-1333", 1],
]

for dataset_name, date_path, fold_id in paths_pix2pix:
    save_path = os.path.join(save_disk, f"pix2pix_fid/{dataset_name}_{date_path}_{fold_id}.npy")
    arr = np.load(save_path)
    print(dataset_name, date_path, fold_id, np.sort(arr)[:k], np.argsort(arr)[:k])
    # print(list(zip(np.argsort(arr)[:k], np.sort(arr)[:k])))
    # for i in np.argsort(arr)[:k]:
    #     g_path = f"F:/TF_LOGS/{dataset_name}/fit/{date_path}/generators/gan_{i}.hdf5"
    #     print(g_path)

for dataset_name, date_path, fold_id in paths_our:
    save_path = os.path.join(save_disk, f"our_fid/{dataset_name}_{date_path}_{fold_id}.npy")
    if not os.path.exists(save_path):
        continue
    arr = np.load(save_path)
    print(dataset_name, date_path, fold_id, np.sort(arr)[:k], np.argsort(arr)[:k])
    # print(list(zip(np.argsort(arr)[:k], np.sort(arr)[:k])))

### pix2pix approach - generate data


In [ ]:
from dataset import DataIterator, load_dataset
import numpy as np
from matplotlib import pyplot as plt
from skimage import color
from glob import glob
from pathlib import Path
from keras import Model, Input, Sequential
from keras.models import load_model
import torch
import os
from tqdm import tqdm
from skimage import io

_ = torch.manual_seed(123)

paths = [
    ["Alizarine", "20230323-110848", 0],
    ["Alizarine", "20230327-224625", 1],
    ["Alizarine", "20230328-021427", 2],
    ["Gavet", "20230323-120633", 0],
    ["Gavet", "20230327-234144", 1],
    ["Gavet", "20230328-030749", 2],
    ["Rotterdam_1000", "20230323-132240", 0],
    ["Rotterdam_1000", "20230328-004342", 1],
    ["Rotterdam_1000", "20230328-095439", 2],
]

save_disk = "D:/"

for dataset_name, date_path, fold_id in paths:
    data_path = f"./datasets/{dataset_name}/folds.json"
    train, test = load_dataset(data_path, True, as_numpy=False)[fold_id]
    dataset_mask = DataIterator(
        train, 32, patch_per_image=32, inv_values=False
    ).get_dataset()
    dataset_test = DataIterator(
        test, 32, patch_per_image=64, inv_values=False
    ).get_dataset()

    images, masks = dataset_test[1], dataset_mask[0]
    validation_data = np.asarray(
        [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in images], dtype=np.uint8
    )

    for k in range(len(validation_data)):
        fp = os.path.join(
            save_disk, f"TF_LOGS/pix2pix/{dataset_name}/dataset_true_fold_{fold_id}/"
        )
        Path(fp).mkdir(exist_ok=True, parents=True)
        io.imsave(
            os.path.join(fp, f"{k}.png"), validation_data[k], check_contrast=False
        )

    for i in tqdm(range(1, 101)):
        g_path = f"F:/TF_LOGS/{dataset_name}/fit/{date_path}/generators/gan_{i}.hdf5"
        generator: Model = load_model(g_path)
        new_mask = generator.predict_on_batch(masks)
        imgs = np.asarray(
            [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in new_mask],
            dtype=np.uint8,
        )

        for k in range(len(imgs)):
            fp = os.path.join(
                save_disk, f"TF_LOGS/pix2pix/{dataset_name}/{date_path}/{i}/"
            )
            Path(fp).mkdir(exist_ok=True, parents=True)
            io.imsave(os.path.join(fp, f"{k}.png"), imgs[k], check_contrast=False)

### our approach - generate data


In [ ]:
from dataset import DataIterator, load_dataset
import numpy as np
from matplotlib import pyplot as plt
from skimage import color
from glob import glob
from pathlib import Path
from keras import Model, Input, Sequential
from keras.models import load_model
import torch
import os
from tqdm import tqdm
from skimage import io

_ = torch.manual_seed(123)


def remove_gaussian_from_model(model: Model) -> Model:
    lay_1 = model.get_layer("conv2d_7")
    lay_2 = model.get_layer("output")
    x = lay_1(model.layers[-4].output)
    x = lay_2(x)
    new_model = Model(inputs=model.input, outputs=x)
    return new_model


paths = [
    ["Rotterdam_1000", "20230330-1333", 1],
    # ["Rotterdam_1000", "20220623-0916", 0],
    # ["Rotterdam_1000", "20220609-1841", 1],
    # ["Rotterdam_1000", "20220609-2031", 2],
    # ["Alizarine", "20220405-2359", 0],
    # ["Alizarine", "20220603-1720", 1],
    # ["Alizarine", "20220603-1850", 2],
    # ["Gavet", "20220429-0021", 0],
    # ["Gavet", "20220609-2213", 1],
    # ["Gavet", "20220609-2319", 2],
]

save_disk = "R:/"
save_true = False

for dataset_name, date_path, fold_id in paths:
    data_path = f"./datasets/{dataset_name}/folds.json"
    train, test = load_dataset(data_path, True, as_numpy=False)[fold_id]
    dataset_mask = DataIterator(
        train, 32, patch_per_image=32, inv_values=False
    ).get_dataset()
    dataset_test = DataIterator(
        test, 32, patch_per_image=64, inv_values=False
    ).get_dataset()

    images, masks = dataset_test[1], dataset_mask[0]

    validation_data = np.asarray(
        [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in images],
        dtype=np.uint8,
    )
    for k in range(len(validation_data)):
        fp = os.path.join(
            save_disk, f"TF_LOGS/our/{dataset_name}/dataset_true_fold_{fold_id}/"
        )
        Path(fp).mkdir(exist_ok=True, parents=True)
        io.imsave(
            os.path.join(fp, f"{k}.png"), validation_data[k], check_contrast=False
        )

    for i in tqdm(range(100)):
        g_path = (
            f"F:/Deep Learning/GAN-CNN-CE/generator/models/{date_path}/model_{i}.h5"
        )

        if not os.path.exists(g_path):
            continue
        
        generator: Model = remove_gaussian_from_model(load_model(g_path))
        new_mask = generator.predict_on_batch([1-masks, np.random.normal(size=masks.shape)])
        imgs = np.asarray(
            [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in new_mask],
            dtype=np.uint8,
        )

        for k in range(len(imgs)):
            fp = os.path.join(save_disk, f"TF_LOGS/our/{dataset_name}/{date_path}/{i}/")
            Path(fp).mkdir(exist_ok=True, parents=True)
            io.imsave(os.path.join(fp, f"{k}.png"), imgs[k], check_contrast=False)

### calculate and save FID

In [ ]:
import os
import subprocess
import re
import numpy as np
import tqdm

save_disk = "R:/TF_LOGS"
paths_pix2pix = [
    # ["Alizarine", "20230323-110848", 0],
    # ["Alizarine", "20230327-224625", 1],
    # ["Alizarine", "20230328-021427", 2],
    # ["Gavet", "20230323-120633", 0],
    # ["Gavet", "20230327-234144", 1],
    # ["Gavet", "20230328-030749", 2],
    # ["Rotterdam_1000", "20230323-132240", 0],
    # ["Rotterdam_1000", "20230328-004342", 1],
    # ["Rotterdam_1000", "20230328-095439", 2],
]

paths_our = [
    ["Rotterdam_1000", "20230330-1333", 1],
    # ["Rotterdam_1000", "20220623-0916", 0],
    # ["Rotterdam_1000", "20220609-1841", 1],
    # ["Rotterdam_1000", "20220609-2031", 2],
    # ["Alizarine", "20220405-2359", 0],
    # ["Alizarine", "20220603-1720", 1],
    # ["Alizarine", "20220603-1850", 2],
    # ["Gavet", "20220429-0021", 0],
    # ["Gavet", "20220609-2213", 1],
    # ["Gavet", "20220609-2319", 2],
]

if False:
    for dataset_name, date_path, fold_id in paths_pix2pix:
        true_path = os.path.join(
            save_disk, f"pix2pix/{dataset_name}/dataset_true_fold_{fold_id}/"
        )
        fids = []
        for i in tqdm.tqdm(range(1, 101)):
            pred_path = os.path.join(save_disk, f"pix2pix/{dataset_name}/{date_path}/{i}/")
            command = (
                f"python -m pytorch_fid {str(pred_path)} {str(true_path)} --device cuda:0"
            )
            arch = subprocess.check_output(command, shell=True)
            fid = float(re.findall("\d+\.\d+", str(arch))[0])
            fids.append(fid)
        save_path = os.path.join(save_disk, f"{dataset_name}_{date_path}_{fold_id}.npy")
        print(save_path, true_path)
        np.save(save_path, np.array(fids))
if True:
    for dataset_name, date_path, fold_id in paths_our:
        true_path = os.path.join(
            save_disk, f"our/{dataset_name}/dataset_true_fold_{fold_id}/"
        )
        fids = []
        for i in tqdm.tqdm(range(100)):
            pred_path = os.path.join(save_disk, f"our/{dataset_name}/{date_path}/{i}/")
            command = (
                f"python -m pytorch_fid {str(pred_path)} {str(true_path)} --device cuda:0"
            )
            arch = subprocess.check_output(command, shell=True)
            fid = float(re.findall("\d+\.\d+", str(arch))[0])
            fids.append(fid)
        save_path = os.path.join(save_disk, f"our_fid/{dataset_name}_{date_path}_{fold_id}.npy")
        print(save_path, true_path)
        np.save(save_path, np.array(fids))

# os.system("shutdown /s /t 30")

In [ ]:
np.save('R:/Rotterdam_1000_20230330-1333_1.npy', fids)

In [ ]:
from tqdm import tqdm
import subprocess
save_disk = "D:/TF_LOGS"
k = 4

paths_pix2pix = [
    ["Alizarine", "20230323-110848", 0, '20230329-2116', 7],
    ["Alizarine", "20230327-224625", 1, '20230329-2120', 7],
    ["Alizarine", "20230328-021427", 2, '20230329-2123', 7],
    ["Gavet", "20230323-120633", 0, '20230329-2127', 9],
    ["Gavet", "20230327-234144", 1, '20230329-2131', 9],
    ["Gavet", "20230328-030749", 2, '20230329-2135', 9],
    ["Rotterdam_1000", "20230323-132240", 0, '20230329-2139', 15],
    ["Rotterdam_1000", "20230328-004342", 1, '20230329-2143', 15],
    ["Rotterdam_1000", "20230328-095439", 2, '20230329-2146', 15],
]

results = []
for dataset_name, date_path, fold_id, models_p, wsize in tqdm(paths_pix2pix):
    command = (
        f"python evaluate.py {dataset_name} {fold_id} segmentation/models/synthetic_pix2pix/{models_p} {wsize}"
    )
    arch = subprocess.check_output(command, shell=True)
    results.append(arch)
    


In [ ]:
results

### TF Benchmark

#### 32x32x3
##### Batch size: 512
PL 100% - 5.84s
PL  90% - 5.85s
PL  80% - 5.85s
PL  70% - 5.89s
PL  60% - 6.07s
PL  50% - 6.20s
##### Batch size: 2048
PL 100% - 3.97s
PL  90% - 3.98s
PL  80% - 4.14s
PL  70% - 3.78s
PL  60% - 3.80s
PL  50% - 4.12s

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from dataset import DataIterator, load_dataset
import numpy as np
from matplotlib import pyplot as plt
from skimage import color
from glob import glob
from pathlib import Path
from keras import Model, Input, Sequential
from keras.models import load_model
import torch
import os
from tqdm import tqdm
from skimage import io
import subprocess
import re
from pytorch_fid.fid_score import calculate_fid_given_paths
_ = torch.manual_seed(123)


def remove_gaussian_from_model(model: Model) -> Model:
    lay_1 = model.get_layer(index=-2)
    lay_2 = model.get_layer("output")
    x = lay_1(model.layers[-4].output)
    x = lay_2(x)
    new_model = Model(inputs=model.input, outputs=x)
    return new_model


paths = [

    ["Gavet", "20230412-1813", 3],
    ["Gavet", "20230412-1829", 3],
    ["Gavet", "20230412-1845", 3],
    ["Gavet", "20230412-1904", 3],
    # ["Alizarine", "20230411-2300", 3],
    # ["Alizarine", "20230410-2152", 3],
    # ["Alizarine", "20230410-2229", 3],
    # ["Alizarine", "20230410-2306", 3],
    # ["Alizarine", "20230410-2338", 3],
    # ["Alizarine", "20230411-0013", 3],
    
    # ["Rotterdam_1000", "20230408-0926", 3],
    # ["Rotterdam_1000", "20230408-1025", 3],
    # ["Rotterdam_1000", "20230408-1124", 3],
    # ["Rotterdam_1000", "20230408-1226", 3],
    # ["Rotterdam_1000", "20230408-1318", 3],
    # ["Rotterdam_1000", "20230408-1413", 3],

    
    # ["Gavet", "20230407-1804", 3],
    # ["Gavet", "20230407-1840", 3],
    # ["Gavet", "20230407-1915", 3],
    # ["Gavet", "20230407-1955", 3],
    # ["Gavet", "20230407-2028", 3],
    # ["Gavet", "20230407-2104", 3],

    
    # ["Gavet", "20220609-2213", 1],
    # ["Gavet", "20230404-1136", 1],
    # ["Gavet", "20230406-1951", 1],
    # ["Gavet", "20230406-2014", 1],
    # ["Gavet", "20230406-2037", 1],
    # ["Gavet", "20230406-2101", 1],
    # ["Gavet", "20230406-2121", 1],
    # ["Gavet", "20230406-2143", 1],
    
    # ["Gavet", "20220609-2213", 0],
    # ["Gavet", "20230330-2249", 0],
    # ["Gavet", "20230330-2355", 0],
    # ["Gavet", "20230331-0947", 0],
    # ["Gavet", "20230331-1023", 0],
    # ["Gavet", "20230331-1046", 0],
    # ["Gavet", "20230331-1240", 0],
]

save_disk = "R:/"
save_true = False
patch_per_image = 32

for dataset_name, date_path, fold_id, in paths:
    data_path = f"./datasets/{dataset_name}/folds.json"
    train, test = load_dataset(data_path, True, as_numpy=False)[fold_id]
    dataset_mask = DataIterator(
        train, 32, patch_per_image=patch_per_image // 2, inv_values=False
    ).get_dataset()
    dataset_test = DataIterator(
        train, 32, patch_per_image=patch_per_image, inv_values=False
    ).get_dataset()

    images, masks = dataset_test[1], dataset_mask[0]

    validation_data = np.asarray(
        [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in images],
        dtype=np.uint8,
    )

    true_path = os.path.join(
        save_disk, f"TF_LOGS/our/{dataset_name}/dataset_true_fold_{fold_id}/"
    )
    for k in range(len(validation_data)):
        Path(true_path).mkdir(exist_ok=True, parents=True)
        io.imsave(
            os.path.join(true_path, f"{k}.png"), validation_data[k], check_contrast=False
        )

    fids = []
    preds_path = []
    for i in tqdm(range(100)):
        g_path = (
            f"F:/Deep Learning/GAN-CNN-CE/logs/{date_path}/generators/model_{i}.h5"
        )

        if not os.path.exists(g_path):
            continue
        try:
            generator: Model = remove_gaussian_from_model(load_model(g_path, compile=False))
        except:
            if i == 0:
                print('cannot remove gaussian')
            generator: Model = load_model(g_path, compile=False)
        
        new_mask = generator.predict_on_batch(
            [1-masks, np.random.normal(size=masks.shape)])
        imgs = np.asarray(
            [color.gray2rgb((im[..., 0] + 1) / 2 * 255) for im in new_mask],
            dtype=np.uint8,
        )
        pred_path = os.path.join(
            save_disk, f"TF_LOGS/our/{dataset_name}/{date_path}/{i}/")
        preds_path.append(pred_path)
        Path(pred_path).mkdir(exist_ok=True, parents=True)
        for k in range(len(imgs)):
            io.imsave(os.path.join(
                pred_path, f"{k}.png"), imgs[k], check_contrast=False)

    command = (
        f"python -m pytorch_fid {str(true_path)} {' '.join(preds_path)} --device cuda:0 --batch-size 128 --num-workers 0 --dims 2048"
    )
    arch = subprocess.check_output(command, shell=True)

    fids_str: str = re.findall("\[(.*?)\]", str(arch))[0]
    fids = [float(val) for val in fids_str.split(',')]
    np.save(f"ablation/{dataset_name}_{date_path}_{fold_id}.npy", fids)
    print(date_path, np.sort(fids)[:5], np.argsort(fids)[:5])



  4%|▍         | 4/100 [00:11<03:03,  1.91s/it]

  5%|▌         | 5/100 [00:11<02:26,  1.54s/it]

100%|██████████| 100/100 [00:53<00:00,  1.86it/s]


20230412-1813 [107.47449734 117.01175454 126.55006418 131.62885976 131.72459966] [ 5 32 22 31  7]


100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


20230412-1829 [ 79.45859402 107.89416214 116.46304104 116.94744639 121.08492495] [47 49 11 12 14]


100%|██████████| 100/100 [00:46<00:00,  2.17it/s]


20230412-1845 [ 93.99914365 126.15721594 130.16915836 138.72851774 143.11731833] [44 38 37 12 39]


100%|██████████| 100/100 [00:52<00:00,  1.90it/s]


20230412-1904 [63.7617624  76.76399027 84.39626781 92.12829262 97.53652036] [39 32 34 19 27]


In [1]:
from glob import glob
import numpy as np
for p in glob('ablation/*'):
    arr = np.load(p)
    print(p, np.sort(arr)[:5], np.argsort(arr)[:5])

"""
ablation\Alizarine_20230411-2300_3.npy [39.93702997 44.59753127 45.04171563 48.35078644 53.75794512] [47 43 42 41 40]
ablation\Alizarine_20230412-0005_3.npy [47.36619009 49.37698269 52.30405712 54.93396577 57.29337334] [43 39 42 49 27]
ablation\Alizarine_20230410-2229_3.npy [53.84472006 56.20737058 57.44760235 63.2678175  66.37898473] [65 71 16 63 58]
ablation\Alizarine_20230410-2306_3.npy [53.60088737 53.86295804 61.31992451 62.08831634 65.60697053] [35  7  8 55 64]
ablation\Alizarine_20230412-0911_3.npy [75.73891031 83.68971065 84.08568789 85.31570024 88.88014644] [12 13 14  9 15]
ablation\Alizarine_20230411-0013_3.npy [58.53753137 65.78021973 66.42443562 67.64621495 79.1568123 ] [16 15 11 10  6]

ablation\Gavet_20230407-1804_3.npy [ 71.0798621   97.16938806  97.89381737 104.10165385 106.05262353] [17  8 26 57  6]
ablation\Gavet_20230407-1840_3.npy [ 84.61955379  87.51672886 103.46925348 104.23312129 110.3638115 ] [85 70 69 84 87]
ablation\Gavet_20230412-1845_3.npy [ 93.99914365 126.15721594 130.16915836 138.72851774 143.11731833] [44 38 37 12 39]
ablation\Gavet_20230407-1955_3.npy [ 87.0183185   94.10139196 101.00991244 102.1441308  104.2853553 ] [14 67 17  9 69]
ablation\Gavet_20230412-1904_3.npy [ 63.7617624  76.76399027 84.39626781 92.12829262 97.53652036] [39 32 34 19 27]
ablation\Gavet_20230407-2104_3.npy [ 113.8845788  115.76705082 116.83317904 119.25901564 121.68402236] [71 84 62 57 59]


ablation\Rotterdam_1000_20230408-0926_3.npy [ 99.98462819 126.05180994 127.89483722 154.5230667  171.53342805] [20 17  5 70 31]
ablation\Rotterdam_1000_20230408-1025_3.npy [112.84785202 122.3684606  131.1417847  137.10432287 141.54142356] [52  8 49 10 48]
ablation\Rotterdam_1000_20230408-1124_3.npy [114.47870366 125.31767138 125.79827823 127.88448793 134.40091213] [91  2 34 42 41]
ablation\Rotterdam_1000_20230408-1226_3.npy [140.69927948 160.05159916 160.07971548 166.76509286 173.0340097 ] [92 84 61 55 56]
ablation\Rotterdam_1000_20230408-1318_3.npy [134.99827381 139.93530297 152.50186911 152.94482703 161.11870304] [48 47 65 39 31]
ablation\Rotterdam_1000_20230408-1413_3.npy [112.35441641 121.48538748 134.97935582 136.32484737 145.55972829] [88 90 25 24 85]
"""

ablation\Alizarine_20230410-2116_3.npy [57.07219175 58.64852124 65.25858369 67.64988741 71.32552164] [18 24 25 20 30]
ablation\Alizarine_20230410-2152_3.npy [40.48346294 44.1504507  46.49580349 47.94317385 49.28306935] [96 78 65 60 67]
ablation\Alizarine_20230410-2229_3.npy [53.84472006 56.20737058 57.44760235 63.2678175  66.37898473] [65 71 16 63 58]
ablation\Alizarine_20230410-2306_3.npy [53.60088737 53.86295804 61.31992451 62.08831634 65.60697053] [35  7  8 55 64]
ablation\Alizarine_20230410-2338_3.npy [39.45736176 45.55028612 45.7483596  46.04107393 46.97180857] [91 95 97 89 63]
ablation\Alizarine_20230411-0013_3.npy [58.53753137 65.78021973 66.42443562 67.64621495 79.1568123 ] [16 15 11 10  6]
ablation\Alizarine_20230411-2300_3.npy [39.93702997 44.59753127 45.04171563 48.35078644 53.75794512] [47 43 42 41 40]
ablation\Alizarine_20230412-0005_3.npy [47.36619009 49.37698269 52.30405712 54.93396577 57.29337334] [43 39 42 49 27]
ablation\Alizarine_20230412-0911_3.npy [75.73891031 83.6

'\nablation\\Alizarine_20230411-2300_3.npy [39.93702997 44.59753127 45.04171563 48.35078644 53.75794512] [47 43 42 41 40]\nablation\\Alizarine_20230412-0005_3.npy [47.36619009 49.37698269 52.30405712 54.93396577 57.29337334] [43 39 42 49 27]\nablation\\Alizarine_20230410-2229_3.npy [53.84472006 56.20737058 57.44760235 63.2678175  66.37898473] [65 71 16 63 58]\nablation\\Alizarine_20230410-2306_3.npy [53.60088737 53.86295804 61.31992451 62.08831634 65.60697053] [35  7  8 55 64]\nablation\\Alizarine_20230412-0911_3.npy [75.73891031 83.68971065 84.08568789 85.31570024 88.88014644] [12 13 14  9 15]\nablation\\Alizarine_20230411-0013_3.npy [58.53753137 65.78021973 66.42443562 67.64621495 79.1568123 ] [16 15 11 10  6]\n\nablation\\Gavet_20230407-1804_3.npy [ 71.0798621   97.16938806  97.89381737 104.10165385 106.05262353] [17  8 26 57  6]\nablation\\Gavet_20230407-1840_3.npy [ 84.61955379  87.51672886 103.46925348 104.23312129 110.3638115 ] [85 70 69 84 87]\nablation\\Gavet_20230407-1915_3.n